In [17]:
from pyspark.sql import *
from pyspark import SparkContext, SparkConf
import json
import numpy as np
import requests
from pyspark.sql import dataframe
from pyspark.sql.types import *

In [29]:
spark = SparkSession.builder \
    .master("local") \
    .appName("HistData") \
    .config("spark.debug.maxToStringFields", "1000") \
    .getOrCreate()


In [26]:
def getResourceDict (json):
    # по листу метрики из json данных возвращает словарь процессов
    list_data = json['data']['result'] # список словарей по процессам
    # предобработка
#     for i in list_data:
#         for j in range(len(i["values"])):
#             i["values"][j][1] = int(i["values"][j][1])
    # values_list = list(map(lambda x: x["values"], list_data))
    resourceTypeDict = dict(map(lambda x: (x["metric"]["resource_type"], x["values"]), list_data))
    # iterObj = iter(resourceTypeDict)
    # print(list(iterObj))
    return resourceTypeDict

In [30]:
# response = requests.get('http://localhost:9090/api/v1/query?query=' + 'PIDMemory' + '[24h]')
# listData = response.json()

with open('C:/Users/Lenovo/Desktop/repoMon/MonitoringSystem/query.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

resourceTypeDict = getResourceDict(json_data)
column_list = list(resourceTypeDict.keys())
print(len(column_list))
data = list(resourceTypeDict.values())
col = ['Time'] + list(resourceTypeDict.keys())

df = None
for el in column_list[:5]:
    df_schema = StructType([StructField("Time", FloatType(), True), StructField(el, DoubleType(), True)])
    if df is None:
        df = spark.createDataFrame(resourceTypeDict[el])
        df = df.toDF("Time", el)
    else:
        df_right = spark.createDataFrame(resourceTypeDict[el]).toDF("Time", el)
        # df_right[el] = df_right[el].withColumn(el, df_right[el].cast('int').alias(el))
        # df_right[el] = df_right[el].cast(FloatType())
        df = df.join(df_right, "Time", "right")

print(f"#### Columns: {len(df.columns)} #### {df.count()}")
df.printSchema()

3324
#### Columns: 6 #### 37
root
 |-- Time: double (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_0: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_1: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_10: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_11: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_2: string (nullable = true)



In [32]:
df.show(5)

+----------------+----------------------------------+----------------------------------+-----------------------------------+-----------------------------------+----------------------------------+
|            Time|AGMService.exe_5208_1604522690.0_0|AGMService.exe_5208_1604522690.0_1|AGMService.exe_5208_1604522690.0_10|AGMService.exe_5208_1604522690.0_11|AGMService.exe_5208_1604522690.0_2|
+----------------+----------------------------------+----------------------------------+-----------------------------------+-----------------------------------+----------------------------------+
|1.604762586663E9|                           7327744|                           3756032|                            3911680|                            3756032|                              5587|
|1.604762241663E9|                           7471104|                           3756032|                            3911680|                            3756032|                              5587|
|1.604762361663E9|  

[[['t1', 'v1'], ['t2', 'v2']], [['t1', 'v2'], ['t2', 'v2']]]
[['t1', 'v1', 'v2'], ['t2', 'v1', 'v2']]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

